In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utils import *

In [8]:
filename = f"data/evaluation_heavyside03_noise_h07.csv"
method = "normal"
df = pd.read_csv(filename)
df_defect = df[df.iloc[:,1] == method+"_defected_data"]
df_typical = df[df.iloc[:,1] == method+"_typical_data"]


if(len(df_defect) > 0):
    FNR = len(df_defect[df_defect.iloc[:,0]==False])/len(df_defect)
else:
    FNR = "No Data"
if(len(df_typical) > 0):
    FPR = len(df_typical[df_typical.iloc[:,0]==True])/len(df_typical)
else:
    FPR = "No Data"

print(f"False Negative Rate {FNR} (type II error)")
print(f"False Positive Rate {FPR} (type I error)")


False Negative Rate 0.032 (type II error)
False Positive Rate 0.108 (type I error)
